In [2]:
import pandas as pd
runways = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/runways.csv')
regions = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/regions.csv')
navaids = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/navaids.csv')
countries = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/countries.csv')
airports = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airports.csv')
airport_frequencies = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-frequencies.csv')
airport_comments = pd.read_csv('/workspace/Airport_Pipeline/Airport_data/airport-comments.csv')

airport_comments


,id,"""threadRef""","""airportRef""","""airportIdent""","""date""","""memberNickname""","""subject""","""body"""
0,508342,82089.0,29339,ESKT,2023-05-01 12:28:13,aviator.bjorn,FREE OF CHARGE,"Well kept and relatively cheap gas, at 30SEK/l..."
1,508327,82088.0,29833,FADD,2023-04-30 03:35:34,Richard747,FADD Flight plan identifier,Dundee identifier FADD is no longer recognised...
2,508326,82087.0,2784,FAGM,2023-04-30 03:33:03,Richard747,Runway identifiers,"Runway identifiers changed from 11R/29L, 17R/3..."
3,508325,82086.0,508324,AU-0599,2023-04-30 01:47:50,adidas,MISSING FROM WEBSITE,MISSING FROM WEBSITE
4,508322,82085.0,508321,AU-0598,2023-04-29 19:46:40,adidas,MISSING FROM WEBSITE,MISSING FROM WEBSITE
...,...,...,...,...,...,...,...,...
15146,5,5.0,1840,CYOW,2007-06-05 02:10:45,NaN,Landing fees,"For private single-engine light aircraft, CAD ..."
15147,4,4.0,1840,CYOW,2007-06-05 02:08:49,NaN,Electrical outlets,The new terminal has lots of electrical outlet...
15148,3,3.0,3920,KTEB,2007-06-05 02:07:04,NaN,U.S. Customs,Customs is pretty easygoing about arrival time...
15149,2,2.0,1891,CYRO,2007-06-05 02:04:37,NaN,IFR,"CYRP does not have an instrument approach, but..."
